# Описание проекта

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 

**Чтобы оценить результаты опроса, оценки обычно делят на три группы:**

9-10 баллов — «cторонники» (англ. promoters);

7-8 баллов — «нейтралы» (англ. passives);

0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».
Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике. 
Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса. 


**Вопросы на которые необходимо ответить:**

Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?

Какие группы пользователей наиболее лояльны к сервису? Какие менее?

Какой общий NPS среди всех опрошенных?

Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

**Импортируем библиотеки:**

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

**Подключаемся к БД:**

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

**Рассмотрим все таблицы изучаемой БД:**

In [8]:
query = """
SELECT * 
FROM user
LIMIT 15;
"""

user = pd.read_sql(query, engine)
user

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10
5,A005O0,5501,42.0,1.0,ANDROID,SMARTPHONE,34,4,6,8,6
6,A0061R,1236,45.0,0.0,ANDROID,SMARTPHONE,55,5,7,8,10
7,A009KS,313,35.0,0.0,ANDROID,SMARTPHONE,28,4,14,5,10
8,A00AES,3238,36.0,1.0,ANDROID,SMARTPHONE,41,4,5,8,10
9,A00F70,4479,54.0,1.0,ANDROID,SMARTPHONE,9,5,8,8,9


In [10]:
query = """
        SELECT * 
        FROM location
        LIMIT 15;
        """
location = pd.read_sql(query, engine)
location

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия
3,4,Барнаул,Россия
4,5,Белгород,Россия
5,6,Брянск,Россия
6,7,Владивосток,Россия
7,8,Владимир,Россия
8,9,Волгоград,Россия
9,10,Волжский,Россия


In [11]:
query = """
        SELECT * 
        FROM age_segment
        LIMIT 15;
        """
age_segment = pd.read_sql(query, engine)
age_segment

,age_gr_id,bucket_min,bucket_max,title
0,1,NaN,15.0,01 до 16
1,2,16.0,24.0,02 16-24
2,3,25.0,34.0,03 25-34
3,4,35.0,44.0,04 35-44
4,5,45.0,54.0,05 45-54
5,6,55.0,64.0,06 55-64
6,7,66.0,NaN,07 66 +
7,8,NaN,NaN,08 n/a


In [12]:
query = """
        SELECT * 
        FROM traffic_segment
        LIMIT 15;
        """
traffic_segment = pd.read_sql(query, engine)
traffic_segment

,tr_gr_id,bucket_min,bucket_max,title
0,1,0.00,0.00,01 0
1,2,0.00,0.01,01 0-0.01
2,3,0.01,0.10,02 0.01-0.1
3,4,0.10,1.00,03 0.1-1
4,5,1.00,5.00,04 1-5
5,6,5.00,10.00,05 5-10
6,7,10.00,15.00,06 10-15
7,8,15.00,20.00,07 15-20
8,9,20.00,25.00,08 20-25
9,10,25.00,30.00,09 25-30


In [13]:
query = """
        SELECT * 
        FROM lifetime_segment
        LIMIT 15;
        """
lifetime_segment = pd.read_sql(query, engine)
lifetime_segment

,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3
3,4,4.0,6.0,04 4-6
4,5,7.0,12.0,05 7-12
5,6,13.0,24.0,06 13-24
6,7,25.0,36.0,07 25-36
7,8,36.0,NaN,08 36+


Посмотрим на нули

In [15]:
query = '''
        SELECT *
        FROM user
        WHERE user_id IS NULL OR
              lt_day IS NULL OR
              gender_segment IS NULL OR
              os_name IS NULL OR
              cpe_type_name IS NULL OR
              location_id IS NULL OR
              age_gr_id IS NULL OR
              tr_gr_id IS NULL OR
              lt_gr_id IS NULL OR
              nps_score IS NULL 
       LIMIT 15;
        '''
user_has_null = pd.read_sql(query, engine)
user_has_null

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A1E59W,4055,NaN,None,ANDROID,SMARTPHONE,13,8,5,8,1
1,A1S6VL,335,40.0,None,ANDROID,TABLET,14,4,8,5,8
2,A28ZDT,4243,NaN,None,ANDROID,SMARTPHONE,56,8,5,8,5
3,A2GLPQ,2354,NaN,None,ANDROID,SMARTPHONE,16,8,7,8,1
4,A3XMNW,41,32.0,None,ANDROID,SMARTPHONE,41,3,4,2,1
5,A41C1K,557,NaN,None,ANDROID,SMARTPHONE,38,8,5,6,3
6,A4590P,94,27.0,None,ANDROID,SMARTPHONE,28,3,6,4,10
7,A48OT7,61,33.0,None,ANDROID,SMARTPHONE,28,3,12,3,9
8,A4T128,76,38.0,None,ANDROID,SMARTPHONE,28,4,7,3,10
9,A513VG,4705,NaN,None,IOS,SMARTPHONE,13,8,24,8,9


**Посмотрим на пропуски**

In [17]:
query = '''
        SELECT COUNT(*)
        FROM user
        WHERE user_id IS NULL OR
              lt_day IS NULL OR
              gender_segment IS NULL OR
              os_name IS NULL OR
              cpe_type_name IS NULL OR
              location_id IS NULL OR
              age_gr_id IS NULL OR
              tr_gr_id IS NULL OR
              lt_gr_id IS NULL OR
              nps_score IS NULL; 
        '''
user_has_null_val = pd.read_sql(query, engine)
user_has_null_val

,COUNT(*)
0,1301


**Формируем sql-запрос, выполняем и сохраняем результат выполнения в DataFrame:**

In [2]:
query = '''
        SELECT user_id,
               lt_day,
               CASE
                    WHEN lt_day <= 365 THEN 'Новый'
                    WHEN lt_day >= 365 THEN 'Старый'
                    END AS is_new, age,
               CASE
                    WHEN gender_segment = 0 THEN 'Мужчина'
                    WHEN gender_sиьegment = 1 THEN 'Женщина'
                    ELSE NULL
               END AS gender_segment, os_name, cpe_type_name, l.country, l.city,
               SUBSTRING(a.title, 4) AS age_segment,
               SUBSTRING(t.title, 4) AS traffic_segment,
               SUBSTRING(lt.title, 4) AS lifetime_segment,
               nps_score,
               CASE
                    WHEN nps_score >= 9 THEN 'сторонники'
                    WHEN nps_score > 6 AND nps_score < 9 THEN 'нейтралы'
                    WHEN nps_score <= 6 THEN 'критики'
               END AS nps_group
         FROM user AS u
         INNER JOIN (
                SELECT location_id, 
                CASE
                    WHEN city = 'СанктПетербург' THEN 'Санкт-Петербург'
                    WHEN city = 'НижнийНовгород' THEN 'Нижний Новгород'
                    WHEN city = 'РостовнаДону' THEN 'Ростов-на-Дону'
                    WHEN city = 'НабережныеЧелны' THEN 'Набережные Челны'
                    WHEN city = 'УланУдэ' THEN 'Улан-Удэ'
                    WHEN city = 'НижнийТагил' THEN 'Нижний Тагил'
                    ELSE city END AS city,
                country
                FROM location
            ) AS l ON u.location_id=l.location_id
         INNER JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
         INNER JOIN traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
         INNER JOIN lifetime_segment AS lt ON u.lt_gr_id = lt.lt_gr_id
         WHERE gender_segment IS NOT NULL AND age IS NOT NULL;
         '''


**В запросе выше также подправил отображение городов для наглядности**

In [22]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,Старый,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,Старый,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники


**Преобразуем в CSV:**

In [ ]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

**Ссылка на дэшборд:** https://public.tableau.com/views/DANPS_PROJECTESENIN/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

**Ссылка на презентацию:** https://drive.google.com/file/d/1fgkV2eFYXo70lBi061U428iJgxYdpqyt/view?usp=sharing